# 限制同時間執行的thread數

In [11]:
import os
import queue as q
import time
import datetime

a = [["E:\\test1.py","work1",7],["E:\\test2.py","work2",15],["E:\\test3.py","work3",3]]

class MyThread(q.threading.Thread):
    def __init__(self, condition):
        q.threading.Thread.__init__(self)
        self.cond = condition

    def run(self):
        print ('{0} start\r'.format(self.getName()))

        while (True):
            path, work_name, delay_time = threadPool.get()
            self.cond.acquire() # lock thread,避免同時用同一個thread執行
            print ("{:%H:%M:%S}".format(datetime.datetime.now()) + ' {0}'.format(self.getName()) + ' is running')
            exe = q.threading.Timer(delay_time, to_do, [path, work_name])
            exe.start()
            time.sleep(0.5) #可省略,只是表達同時間有2個thread可執行queue中的工作
            self.cond.release() #派工完,解thread lock
            threadPool.task_done() #註記此工作已做

def to_do(program, variable):
    execute = "python " + program + " " + variable
    print("{:%H:%M:%S}".format(datetime.datetime.now()) + " run " + program + " variable = " + variable)
    os.system(execute)

threadPool = q.Queue(0) #建一個沒有限制可放多少數量的queue
condition = q.threading.Condition() # OS lock機制

#只用2個thread執行所有工作
for i in range(2):
    MyThread(condition).start()

#將list所有工作放到queue中
for i in a:
    threadPool.put(i)

threadPool.join() #得到task done的訊息,將queue中的位置空出來讓後面的工作項目可以補上

Thread-36 start
Thread-37 start
23:02:54 Thread-36 is running
23:02:54 Thread-37 is running
23:02:55 Thread-36 is running
23:02:58 run E:\test3.py variable = work3
23:03:01 run E:\test1.py variable = work1
23:03:09 run E:\test2.py variable = work2


# 精簡版↓

In [117]:
import queue as q
import datetime
import os

a = [["E:\\test1.py","work1",5],["E:\\test2.py","work2",10],["E:\\test3.py","work3",3]]

with open(r'E:\\test.txt','a') as f:
    f.write("{:%H:%M:%S}".format(datetime.datetime.now()) + " start\n")

def to_do(program, variable):
    excute = "python " + program + " " + variable
    os.system(excute)

for i in a:
    t = q.threading.Timer(i[2], to_do, [i[0],i[1]])
    t.start()

a.clear()

# 原始版↓

In [102]:
import threading
import datetime

a = [[5,"work1"],[10,"work2"],[3,"work3"]]
c = 0
work_scheduler = []

def to_do():
    print("{:%H:%M:%S}".format(datetime.datetime.now()) + " do " + work_scheduler[0])
    work_scheduler.pop(0)

print("{:%H:%M:%S}".format(datetime.datetime.now()) + " start")

for i in sorted(a):
    work_scheduler.append(i[c+1])
    t = threading.Timer(i[c], to_do)
    t.start()

20:55:19 start
20:55:22 do work3
20:55:24 do work1
20:55:29 do work2
